In [1]:
import os
import numpy as np
from sklearn.metrics import confusion_matrix
from spoon_detection_model import my_spoon_detection_model
import tensorflow as tf
import tensorflow as tf

assert tf.__version__.startswith('2')

IMAGE_SIZE = 96
BATCH_SIZE = 32

BASE_DIR = os.path.join(os.getcwd(), 'spoon-training-images-converted/spoon')

# Load pre-trained model
model = tf.keras.models.load_model('trained_models/spoon_detection_model.h5')

'''
# Create data generator for test data
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    BASE_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    shuffle=False)
'''

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=.1,
    horizontal_flip=True,
    validation_split=0.1,
    rescale=1. / 255)
train_generator = datagen.flow_from_directory(
    BASE_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training',
    color_mode='grayscale')
val_generator = datagen.flow_from_directory(
    BASE_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation',
    color_mode='grayscale')
print(train_generator.class_indices)

# Evaluate model on training data
_, train_acc = model.evaluate(train_generator)
print('Training Accuracy:', train_acc)

# Evaluate model on validation data
_, val_acc = model.evaluate(val_generator)
print('Validation Accuracy:', val_acc)

# Evaluate model on test data
test_loss, test_acc = model.evaluate(train_generator)
print('Test Accuracy:', test_acc)

# Compute FRR and FPR
y_pred = model.predict(train_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = train_generator.classes
cm = confusion_matrix(y_true, y_pred_classes)
tn, fp, fn, tp = cm.ravel()
frr = fn / (fn + tp)
fpr = fp / (fp + tn)
print('FRR:', frr)
print('FPR:', fpr)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'c:\\Users\\Naseer\\OneDrive\\Desktop\\Spring-2023\\ECGR5127-Machine_Learning_for_IoT\\Project_1\\spoon_detection_model\\spoon-training-images-converted/person'

In [11]:
import tensorflow as tf

# Load your trained model
model = tf.keras.models.load_model('spoon_detection_model.h5')

# Create input and output signatures
input_shape = (1, 96, 96, 1)
input_signature = [
    tf.TensorSpec(shape=input_shape, dtype=tf.float32, name='input')
]
output_signature = [
    tf.TensorSpec(shape=(None, 2), dtype=tf.float32, name='output')
]

# Profile the model to find number of MACs
profiler = tf.profiler.experimental.Profiler('/tmp/profile')
profiler.start()
result = model(tf.ones(input_shape))
profiler.stop()

# Print the number of MACs
stats = profiler.profile_operations(profiled_tensors=[result.op])
macs = sum([stat.total_float_ops for stat in stats])
print(f"Number of MACs: {macs}")


AttributeError: module 'tensorflow.profiler.experimental' has no attribute 'Profiler'